In [1]:
import pandas as pd
from sklearn.pipeline import Pipeline

In [2]:
Dataset=pd.read_csv('CKD.csv')

In [3]:
Dataset.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,2.0,76.459948,c,3,0,normal,abnormal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,yes,no,yes
1,3.0,76.459948,c,2,0,normal,normal,notpresent,notpresent,148.112676,...,34.000000,12300.000000,4.705597,no,no,no,yes,poor,no,yes
2,4.0,76.459948,a,1,0,normal,normal,notpresent,notpresent,99.000000,...,34.000000,8408.191126,4.705597,no,no,no,yes,poor,no,yes
3,5.0,76.459948,d,1,0,normal,normal,notpresent,notpresent,148.112676,...,38.868902,8408.191126,4.705597,no,no,no,yes,poor,yes,yes
4,5.0,50.000000,c,0,0,normal,normal,notpresent,notpresent,148.112676,...,36.000000,12400.000000,4.705597,no,no,no,yes,poor,no,yes


In [4]:
Dataset=pd.get_dummies(Dataset,dtype=int,drop_first=True)

In [5]:
Dataset.columns

Index(['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes', 'classification_yes'],
      dtype='object')

In [13]:
Independent=Dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv',
       'wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 'rbc_normal', 'pc_normal',
       'pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes',
       'appet_yes', 'pe_yes', 'ane_yes']]
Dependent=Dataset[['classification_yes']]

In [15]:
from sklearn.model_selection import GridSearchCV

from sklearn.naive_bayes import CategoricalNB

param_grid = {
    'alpha': [0.0, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
    'fit_prior': [True, False],
    'min_categories': [ 2, 3, 5]
 
   
}

# Set up GridSearchCV
grid = GridSearchCV(CategoricalNB(), param_grid, refit=True, verbose=3, n_jobs=-1, scoring='f1_weighted')

grid.fit(Independent,Dependent)

Fitting 5 folds for each of 42 candidates, totalling 210 fits


c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:1051: UserWarning: One or more of the test scores are non-finite: [nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan nan
 nan nan nan nan nan nan]
  warnings.warn(
c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Gajalakshmi G\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\naive_bayes.py:1504: RuntimeWarning: divide by zero encountered in log
  np.log(smoothed_cat_count) - np.log(smoothed_class_count.reshape(-1, 1))


GridSearchCV(estimator=CategoricalNB(), n_jobs=-1,
             param_grid={'alpha': [0.0, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0],
                         'fit_prior': [True, False],
                         'min_categories': [2, 3, 5]},
             scoring='f1_weighted', verbose=3)

In [11]:
re=grid.cv_results_

In [16]:
grid_Pred=grid.predict(Independent)

In [18]:
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(grid_Pred,Dependent)
print(cm)

[[150   0]
 [  0 249]]


In [19]:
from sklearn.metrics import classification_report
clf_report=classification_report(grid_Pred,Dependent)
print(clf_report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       150
           1       1.00      1.00      1.00       249

    accuracy                           1.00       399
   macro avg       1.00      1.00      1.00       399
weighted avg       1.00      1.00      1.00       399



In [20]:
from sklearn.metrics import f1_score
f1_macro=f1_score(grid_Pred,Dependent)
print("The f1 sore of the best parameter{}:".format(grid.best_params_),f1_score)

The f1 sore of the best parameter{'alpha': 0.0, 'fit_prior': True, 'min_categories': 2}: <function f1_score at 0x000002C3BAA4A160>


In [21]:
from sklearn.metrics import roc_auc_score
roc_auc_score(Dependent,grid.predict_proba(Independent)[:,1])

1.0

In [22]:
table=pd.DataFrame.from_dict(re)
print(table)

    mean_fit_time  std_fit_time  mean_score_time  std_score_time param_alpha  \
0        0.031663      0.003610         0.016264        0.004613         0.0   
1        0.023807      0.010591         0.015962        0.003993         0.0   
2        0.014384      0.002729         0.005961        0.002724         0.0   
3        0.015849      0.001583         0.006776        0.004049         0.0   
4        0.018376      0.002536         0.007535        0.002796         0.0   
5        0.024279      0.010337         0.009892        0.005343         0.0   
6        0.015146      0.003017         0.011822        0.006038         0.1   
7        0.015399      0.002889         0.010847        0.006439         0.1   
8        0.014214      0.001451         0.007581        0.003610         0.1   
9        0.014573      0.001360         0.008842        0.005802         0.1   
10       0.014513      0.001654         0.009060        0.004297         0.1   
11       0.014074      0.001354         